# 본 페이지의 목표

-gensim에서 제공하는 fasttext 모델에 없는 단어를 학습시킨다.

-없는 단어를 학습하기 위해 크롤링을 하여 context를 모은다.

-크롤링한 context를 fasttext에 학습시킨다.

In [1]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import codecs
%matplotlib inline

In [42]:
list_items = pd.read_csv('C:/Users/admin/Desktop/NLP/description/skipgram/오타확인/total_typo_check_list.txt',header = None, delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
list_items = list_items[0]
print(list_items[0:3])

0    STABILISATION
1           MOOHAN
2            INLAY
Name: 0, dtype: object


# 기존에 존재하지 않은 단어 크롤링! 

크롤링을 통해 source,target description에 미포함된 단어의 context를 만든다.

만드는 규칙 따위 없다. 그냥 단어에 상관없이 문장을 다 더할것이다. 

In [93]:
#527 index까지 탐색했음 
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re # 한글 분리
hangul = re.compile('[^가-힣]+')
english = re.compile('[^a-zA-Z0-9.,:-]+')
not_english = re.compile('[a-zA-Z0-9.,]+')
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument('--dns-prefetch-disable')
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14931")
#크롬 드라이버 경로를 위치시킨다
driver = webdriver.Chrome('C:/Users/admin/Desktop/NLP/chromedriver_v71/chromedriver', options=options)
# crawl_txt = []
driver.implicitly_wait(3)
# ex =['COFLEXINTERSPINOUSIMPLANTS']#list_items
for idx,search_item in enumerate(list_items[527:]):
    driver = webdriver.Chrome('C:/Users/admin/Desktop/NLP/chromedriver_v71/chromedriver', options=options)
    driver.implicitly_wait(3)
    url = "https://www.google.com"
    driver.get(url)
    elem = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input')
    elem.send_keys(str(search_item))
    elem.submit()
    driver.implicitly_wait(5) # 브라우저 검색 최소 지연시간 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    notices = soup.select('#fprsl > em')
    no_result = soup.select('#topstuff > div > div > p') 
    div_group=[]

    
    if(len(no_result)>0):
        print('검색결과 없음')
        continue
                             
    if(len(notices)>0): # 해당 단어의 예상 추천 단어 추천단어로 재검색 
        print('추천 단어로 재검색: ')
        driver = webdriver.Chrome('C:/Users/admin/Desktop/NLP/chromedriver_v71/chromedriver', options=options)
        driver.implicitly_wait(5)
        url = "https://www.google.com"
        driver.get(url)
        driver.implicitly_wait(5) # 브라우저 검색 최소 지연시간   
        elem = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input')
        elem.send_keys(str(notices[0].text))
        search_item = str(notices[0].text)
        elem.submit()
        driver.implicitly_wait(3) # 브라우저 검색 최소 지연시간    
    div_group=soup.find_all("span", {"class": "st"}) 
    for div in div_group: 
        try:
            a =str(div.get_text()).replace("...","")
#             temp2 = english.sub(' ',a ) #영어만 추출
            a_temp = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ_!』\\‘|\(\)\[\]\<\>`\'…》]', '', a)
            a_temp=re.sub('\W+','', a_temp)#두번에 걸쳐 특수문자를 제거 -> 그 후에도 남아있는 것은 영어가 아닌 단어라고 생각
            pattern = re.compile(r'\s+')
            a_temp = re.sub(pattern, '', a_temp)
            temp3 =not_english.sub('',a_temp)#영어가 아닌 단어 추출

            
            if len(temp3) > 0:#영어가 아닌 단어가 존재하면 넘어간다.
                continue
#             print(temp3)
            a = re.sub('[=+#/\?^;$.@*\"※~&%ㆍ_!』·“”\\‘|\(\)\[\]\<\>`\'…》]', '', a)
#             print(a)# 영어만 존재할 경우 append
            print('현재 크롤링된 index:%d , 현재 단어:%s '%(idx+527,str(search_item)))
            print('영어로된 크롤링 문장: ',a)
            crawl_txt.append(str(a))
                
        except NoSuchElementException:
                print("찾고자하는 검색결과가 하나도 없어서 에러남 ")
                
    driver.implicitly_wait(3)
    


현재 크롤링된 index:212 , 현재 단어:INFOMED 
영어로된 크롤링 문장:  Infomed is leader in extracorporeal treatment as plasmatherapies and kidney failure Infomed is specialized in blood purification device
현재 크롤링된 index:212 , 현재 단어:INFOMED 
영어로된 크롤링 문장:  Infofly is a renowned travel agency specialized in the tourism and travel industry Infofly offers the most extensive range of travel opportunities for all kinds of 
현재 크롤링된 index:212 , 현재 단어:INFOMED 
영어로된 크롤링 문장:  Taller de Wordpress Laboratorio Docente Infomed  12 - 14 febrero 2019  Software de presentaciones Prezi Laboratorio Docente Infomed  18 febrero 2019 
현재 크롤링된 index:212 , 현재 단어:INFOMED 
영어로된 크롤링 문장:  Labxpert by INFOMED is an Information System for Analytical Laboratories which gives you the ability of data management of your Laboratory in electronic 
현재 크롤링된 index:212 , 현재 단어:INFOMED 
영어로된 크롤링 문장:  Learn about working at Infomed International for Events Join LinkedIn today for free See who you know at Infomed International for Events, leverage yo

현재 크롤링된 index:218 , 현재 단어:TLC55 
영어로된 크롤링 문장:  Sorry, but this product isnt available for Regional EPC - NA TLC55 Product Name PROXIMATE® Linear Cutters Order product shoppingcart Product 
현재 크롤링된 index:218 , 현재 단어:TLC55 
영어로된 크롤링 문장:  TLC55 View specs TLC75 View specs TCT75 View specs TLC10 View specs TCT10 View specs PROXIMATE Linear Cutter Optimal Performance Guide
현재 크롤링된 index:218 , 현재 단어:TLC55 
영어로된 크롤링 문장:  In Stock Ships Today 8500 box of 1 - Ethicon Endo Surgery  TLC55 - Ethicon Proximate Linear Cutter: 55mm
현재 크롤링된 index:218 , 현재 단어:TLC55 
영어로된 크롤링 문장:  ETHICON TLC55: EACH IN-STOCK ORDERS RECEIVED BEFORE 4PM EST SHIPPED SAME DAY ETHICON PROXIMATE LINEAR CUTTER : BLUE 
현재 크롤링된 index:218 , 현재 단어:TLC55 
영어로된 크롤링 문장:  Linear Cutter Proximate® Squeeze Handle Titanium Staples Blue Staple Loading Unit J  J Healthcare Systems TLC55
현재 크롤링된 index:218 , 현재 단어:TLC55 
영어로된 크롤링 문장:  Amazoncom: Ethicon TLC55 Proximate Linear Cutter, Regular Tissue, 55 mm Staple Line Length, Blue Pack of 3:

현재 크롤링된 index:225 , 현재 단어:MEDICLORE 
영어로된 크롤링 문장:  2017 9 26 - The investigators planned double blind prospective study of Thermo-sensitive Adhesion Barrier MEDICLORE to reveal anti-adhesive effect 
현재 크롤링된 index:225 , 현재 단어:MEDICLORE 
영어로된 크롤링 문장:  2016 10 20 - Clinical Trial to Evaluate the Efficacy and Safety of Adhesion Barrier Mediclore Versus no Treatment in Patients With Total Thyroidectomy 
현재 크롤링된 index:225 , 현재 단어:MEDICLORE 
영어로된 크롤링 문장:  Mediclore New Generation of Anti-adhesion Superior barrier with prompt gelation in situ Advanced mucous membrane adhesiveness Antibacterial and local 
현재 크롤링된 index:225 , 현재 단어:MEDICLORE 
영어로된 크롤링 문장:  Anti-adhesive Effect and Safety of Thermo-sensitive Adhesion Barrier MEDICLORE by Using Double-blind Test in Thyroid Surgery Anti-adhesive Effect and 
현재 크롤링된 index:225 , 현재 단어:MEDICLORE 
영어로된 크롤링 문장:  Mediclore This product is a deep cavity wound covering material and mainly used for reducing adhesion after spinal operation Composition Poloxa

현재 크롤링된 index:231 , 현재 단어:THERMODYNAMIC 
영어로된 크롤링 문장:  Thermodynamics is the branch of physics that has to do with heat and temperature and their relation to energy and work The behavior of these quantities is 
현재 크롤링된 index:231 , 현재 단어:THERMODYNAMIC 
영어로된 크롤링 문장:  The four laws of thermodynamics define fundamental physical quantities temperature, energy, and entropy that characterize thermodynamic systems at thermal 
현재 크롤링된 index:231 , 현재 단어:THERMODYNAMIC 
영어로된 크롤링 문장:  Thermodynamics, science of the relationship between heat, work, temperature, and energy In broad terms, thermodynamics deals with the transfer of energy 
현재 크롤링된 index:231 , 현재 단어:THERMODYNAMIC 
영어로된 크롤링 문장:  Thermodynamics is a branch of physics which deals with the energy and work of a system It was born in the 19th century as scientists were first discovering how 
현재 크롤링된 index:231 , 현재 단어:THERMODYNAMIC 
영어로된 크롤링 문장:  In order to avoid confusion, scientists discuss thermodynamic values in reference to a system and 

현재 크롤링된 index:238 , 현재 단어:DESYNE 
영어로된 크롤링 문장:  Introducing the Novolimus-eluting DESyne stent - the next generation drug eluting stent with improved performance and superior clinical efficacy and safety1
현재 크롤링된 index:238 , 현재 단어:DESYNE 
영어로된 크롤링 문장:  DESyne® X2 NOVOLIMUS™ Eluting Coronary Stent System Stent Design Exceptional Deliverability Ultra Thin DESyne Stent • Low profile stent delivery 
현재 크롤링된 index:238 , 현재 단어:DESYNE 
영어로된 크롤링 문장:  The Elixir DESyne BD Novolimus-Eluting Coronary Stent System represents a significant advancement in biodegradable coating technology, eliminating the 
현재 크롤링된 index:238 , 현재 단어:DESYNE 
영어로된 크롤링 문장:  Advanced DrugPolymer Coating Technology -Thin polymer and drug matrix coating without the need for a primer coating -Superior DES clinical performance 
현재 크롤링된 index:238 , 현재 단어:DESYNE 
영어로된 크롤링 문장:  Desyne specializes in high-performance web applications, design, development and performance optimization
현재 크롤링된 index:238 , 현재 단어:DESYNE 
영어로된 크롤링 문장: 

현재 크롤링된 index:245 , 현재 단어:ASPIRATION 
영어로된 크롤링 문장:  Aspiration or aspirations may refer to: Contents 1 Linguistics 2 Medicine 3 Other uses 4 See also Linguisticsedit Aspirated consonant, a plosive or fricative 
현재 크롤링된 index:245 , 현재 단어:ASPIRATION 
영어로된 크롤링 문장:  A financial firm you can fall in love with: banking and investing that puts you and your conscience first
현재 크롤링된 index:245 , 현재 단어:ASPIRATION 
영어로된 크롤링 문장:  2017 4 3 - Aspiration is common, but it can increase your risk for health conditions such as pneumonia Learn what causes aspiration and how to prevent 
현재 크롤링된 index:245 , 현재 단어:ASPIRATION 
영어로된 크롤링 문장:  Aspiration definition is - a strong desire to achieve something high or great —usually plural How to use aspiration in a sentence Synonym Discussion of 
현재 크롤링된 index:245 , 현재 단어:ASPIRATION 
영어로된 크롤링 문장:  Aspiration definition, a strong desire, longing, or aim ambition: intellectual aspirations See more
현재 크롤링된 index:245 , 현재 단어:ASPIRATION 
영어로된 크롤링 문장:  Synonyms for aspir

현재 크롤링된 index:252 , 현재 단어:DERMATIX 
영어로된 크롤링 문장:  Dermatix is a transparent, self-drying silicone gel that improves the appearance of scars  Softens and flattens raised scars  Fades red or dark scars  Relieves 
현재 크롤링된 index:252 , 현재 단어:DERMATIX 
영어로된 크롤링 문장:  Buy Dermatix Ultra - Advanced Scar Formula Innovative CPX Technology  Unique Vitamin C Ester 15g on Amazoncom ✓ FREE SHIPPING on qualified orders
현재 크롤링된 index:252 , 현재 단어:DERMATIX 
영어로된 크롤링 문장:  Dermatix® Ultra is an advanced scar formula that is proven effective in the management of scars It has the innovative CPX cyclopentasiloxane technology for 
현재 크롤링된 index:252 , 현재 단어:DERMATIX 
영어로된 크롤링 문장:  I used this for my daughters forehead scar after she bumped into an sharp iron corner and got stitches I used it everyday twice on the stitch scars, and 
현재 크롤링된 index:252 , 현재 단어:DERMATIX 
영어로된 크롤링 문장:  2005 6 27 - Dermatix gel is applied from a tube twice daily for at least two months It is thought to improve hydration to the scar, 

현재 크롤링된 index:258 , 현재 단어:30MMHG 
영어로된 크롤링 문장:  Buy SB SOX Compression Socks 20-30mmHg for Men  Women - Best Stockings for Running, Medical, Athletic, Edema, Diabetic, Varicose Veins, Travel, 
현재 크롤링된 index:258 , 현재 단어:30MMHG 
영어로된 크롤링 문장:  GRADUATED COMPRESSION  FASTER MUSCLE RECOVER - Our graduation compression technology 20-30mmHg provides the ideal compression to 
현재 크롤링된 index:258 , 현재 단어:30MMHG 
영어로된 크롤링 문장:  Most commonly compression support stockings come in mild 8-15 mmHg, Medium 15-20 mmHg, Firm 20-30 mmHg, X-Firm 30-40 mmHg gradient 
현재 크롤링된 index:258 , 현재 단어:30MMHG 
영어로된 크롤링 문장:  Jobst Ultrasheer 20-30 mmHg Knee High Firm Compression Stockings Fashion and efficacy rolled all into one Jobst® UltraSheer is the sheerest compression 
현재 크롤링된 index:258 , 현재 단어:30MMHG 
영어로된 크롤링 문장:  Jobst Relief 20-30 mmhg Knee High Firm Compression Stockings Jobst Relief Closed Toe Compression Stockings Gradient Compression Therapy, at a price 
현재 크롤링된 index:258 , 현재 단어:30MMHG 
영어로된 크롤링 문장:  Exp

현재 크롤링된 index:265 , 현재 단어:hypersomnias 
영어로된 크롤링 문장:  Hypersomnia, or hypersomnolence, is a neurological disorder of excessive time spent sleeping or excessive sleepiness It can have many possible causes and 
현재 크롤링된 index:265 , 현재 단어:hypersomnias 
영어로된 크롤링 문장:  Classification of Hypersomnias  Idiopathic hypersomnia patients frequently, but not always, sleep for very long amounts of time Patients with idiopathic 
현재 크롤링된 index:265 , 현재 단어:hypersomnias 
영어로된 크롤링 문장:  2017 10 29 - WebMD looks at the condition called hypersomnia, characterized by excessive daytime sleepiness
현재 크롤링된 index:265 , 현재 단어:hypersomnias 
영어로된 크롤링 문장:  UpToDate, electronic clinical resource tool for physicians and patients that provides information on Adult Primary Care and Internal Medicine, Allergy and 
현재 크롤링된 index:265 , 현재 단어:hypersomnias 
영어로된 크롤링 문장:  Central hypersomnias are diseases manifested in excessive daytime sleepiness EDS not caused by disturbed nocturnal sleep or misaligned circadian rhythms 
현재 

TimeoutException: Message: timeout
  (Session info: headless chrome=71.0.3578.98)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)


In [90]:
list_items[212]

'INFOMED'

In [24]:
with codecs.open("C:/Users/admin/Desktop/NLP/description/skipgram/train_source_target_outdata.txt", 'w', "utf-8") as f:
    for data in crawl_txt:
        temp = str(data)+'\n'
        f.write(temp)

679
679
